In [1]:
#import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px
import numpy as np

Explo

In [2]:
tablelist=["name.basics.tsv.gz", "title.akas.tsv.gz", "title.basics.tsv.gz", "title.crew.tsv.gz", "title.episode.tsv.gz", "title.principals.tsv.gz", "title.ratings.tsv.gz", "TMDB.csv"]

In [3]:
infodict={}
for tableid in tablelist:
    print()
    PKey=[]
    PKeysize=[]
    FKey=[]
    FKeysize=[]
    PKD=[]
    FKD=[]
    PFix=[]
    if tableid[-3:]=='.gz':
        dfl=pl.read_csv(tableid, separator="\t", quote_char=None, null_values="\\N")
    elif tableid[-3:]=='csv':
        dfl=pl.read_csv(tableid, low_memory=False, ignore_errors=True, separator=",", truncate_ragged_lines=True)
    type_dict={dfl.columns[i] : dfl.dtypes[i] for i in range(len(dfl.columns))}
    print(f'Table "{tableid}" has Rows_MLN = {round(dfl.shape[0]/1000000,1)} and columns = {dfl.shape[1]} :\n')
    for ky, val in type_dict.items():
        if True:
            try :
                space_contains = round(dfl[ky].str.split(by=" ").map_elements(lambda x: len(x), return_dtype=pl.Int16).mean(),2)
            except :
                True
            print(f'Column "{ky}" Type = "{val}" # ', end='')             
            if ((str(val) == 'String') & (space_contains<1.01)):
                dfl=dfl.with_columns_seq(pl.col(str(ky)).str.replace_all(r"\x5D|\x5B|\x22",""))
                flag_explode = round(dfl[ky].str.split(by=",").map_elements(lambda x: len(x), return_dtype=pl.Int16).mean(),2)
                if flag_explode>1.2:
                    unique_explode=list(dfl.filter(pl.col(ky).is_not_null())[ky].explode().unique())
                    prefix=dfl.filter(pl.col(ky).is_not_null())[ky].explode().unique().map_elements(lambda x: x[0:2], return_dtype=pl.String)
                    print(f'/!\ List detected with ratio {flag_explode*100}% /!\ Unique_MLN = {round(len(unique_explode)/1000000,1)} # Unique_pcts = {round(len(unique_explode)/dfl.shape[0]*100,0)}% # Empty_pcts = {round(dfl[ky].is_null().mean()*100,0)}% # ', end='')
                    if ((round(len(unique_explode)/1000000,1)>0.1) and ((round((len(unique_explode)/dfl.shape[0])*100,0)<101) ^ (round((len(unique_explode)/dfl.shape[0])*100,0)>99))):
                        FKey.append(ky+'_exploded')
                        FKeysize.append(round(len(unique_explode)/1000000,1))
                        FFixD='_'
                        if ((prefix.value_counts().sort(by='count', descending=True)[0,1]) > 0.95*prefix.count()):
                            print(f" Prefix detected = {prefix.value_counts().sort(by='count', descending=True)[0,0]} # ", end='')
                            PFix.append(prefix.value_counts().sort(by='count', descending=True)[0,0])
                            FFixD+=prefix.value_counts().sort(by='count', descending=True)[0,0]+'_'
                        FKD.append(tableid+'_'+ky+'-exploded'+FFixD+str(round(len(unique_explode)/1000000,1)))
                else:
                    prefix=dfl.filter(pl.col(ky).is_not_null())[ky].map_elements(lambda x: x[0:2], return_dtype=pl.String)
                    PFixD='_'
                    FFixD='_'
                    if round((len(dfl[ky].unique())/dfl.shape[0])*100,0)>99:
                        PKey.append(ky)
                        PKeysize.append(round(len(dfl[ky].unique())/1000000,1))
                        if ((prefix.value_counts().sort(by='count', descending=True)[0,1]) > 0.95*prefix.count()):
                            PFixD+=prefix.value_counts().sort(by='count', descending=True)[0,0]+'_'
                        PKD.append(tableid+'_'+ky+PFixD+str(round(len(dfl[ky].unique())/1000000,1)))
                    if ((round(len(dfl[ky].unique())/1000000,1)>0.1) and ((round((len(dfl[ky].unique())/dfl.shape[0])*100,0)<101) ^ (round((len(dfl[ky].unique()))/dfl.shape[0]*100,0)>99))):
                        FKey.append(ky)
                        FKeysize.append(round(len(dfl[ky].unique())/1000000,1))
                        if ((prefix.value_counts().sort(by='count', descending=True)[0,1]) > 0.95*prefix.count()):
                            FFixD+=prefix.value_counts().sort(by='count', descending=True)[0,0]+'_'
                        FKD.append(tableid+'_'+ky+FFixD+str(round(len(dfl[ky].unique())/1000000,1)))
                    print(f'Unique_MLN = {round(len(dfl[ky].unique())/1000000,1)} # Unique_pcts = {round(len(dfl[ky].unique())/dfl.shape[0]*100,0)}% # Empty_pcts = {round(dfl[ky].is_null().mean()*100,0)}% # ', end='')
                    if ((prefix.value_counts().sort(by='count', descending=True)[0,1]) > 0.95*prefix.count()):
                        print(f" Prefix detected = {prefix.value_counts().sort(by='count', descending=True)[0,0]} # ", end='')
                        PFix.append(prefix.value_counts().sort(by='count', descending=True)[0,0])
            else:
                print(f'Unique_MLN = {round(len(dfl[ky].unique())/1000000,1)} # Unique_pcts = {round(len(dfl[ky].unique())/dfl.shape[0]*100,0)}% # Empty_pcts = {round(dfl[ky].is_null().mean()*100,0)}% # ', end='')
                
            print()
        # elif tableid[-3:]=='csv':
        #     if str(val) == 'String':
        #         dfl=dfl.with_columns_seq(pl.col(str(ky)).str.replace_all(r"\x5D|\x5B|\x22",""))
        #     print(f'Colonne "{ky}" Type = "{val}" Unique_M = {round(len(dfl[ky].unique())/1000000,1)} Unique_pcts = {round(len(dfl[ky].unique())/len(dfl[ky])*100,0)}% Empty_pcts = {round(dfl[ky].is_null().mean()*100,0)}%', end='')
        #     if str(val) == 'ing':
        #         flag_explode = round(dfl[ky].str.split(by=",").map_elements(lambda x: len(x), return_dtype=pl.Int16).mean(),1)
        #         print(f' List Length = {flag_explode}', end='')
        #     else:
        #         print()
    print(f"Prefix detected : {PFix}")
    print(f"Primary Key is {PKey} has size_MLN {PKeysize}")
    print(f"Foreign Key could be {FKey} has size_MLN {FKeysize}")
    display(dfl.sample(6).transpose)
    infodict.update({tableid : {'name' : tableid, 'lines' : int(round(dfl.shape[0]/1000000,1)), 'cols' : dfl.shape[1], 'PKey' : { 'id' : PKey, 'Size' : PKeysize },  'FKey' : { 'id' : FKey, 'Size' : FKeysize }, 'PKD' : { 'id' : PKD }, 'FKD' : { 'id' : FKD }}})

   


Table "name.basics.tsv.gz" has Rows_MLN = 13.9 and columns = 6 :

Column "nconst" Type = "String" # Unique_MLN = 13.9 # Unique_pcts = 100.0% # Empty_pcts = 0.0% #  Prefix detected = nm # 
Column "primaryName" Type = "String" # Unique_MLN = 10.7 # Unique_pcts = 77.0% # Empty_pcts = 0.0% # 
Column "birthYear" Type = "Int64" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 95.0% # 
Column "deathYear" Type = "Int64" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 98.0% # 
Column "primaryProfession" Type = "String" # /!\ List detected with ratio 138.0% /!\ Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 19.0% # 
Column "knownForTitles" Type = "String" # /!\ List detected with ratio 185.0% /!\ Unique_MLN = 5.8 # Unique_pcts = 42.0% # Empty_pcts = 11.0% #  Prefix detected = tt # 
Prefix detected : ['nm', 'tt']
Primary Key is ['nconst'] has size_MLN [13.9]
Foreign Key could be ['knownForTitles_exploded'] has size_MLN [5.8]


<bound method DataFrame.transpose of shape: (6, 6)
┌────────────┬────────────────────┬───────────┬───────────┬───────────────────┬────────────────────┐
│ nconst     ┆ primaryName        ┆ birthYear ┆ deathYear ┆ primaryProfession ┆ knownForTitles     │
│ ---        ┆ ---                ┆ ---       ┆ ---       ┆ ---               ┆ ---                │
│ str        ┆ str                ┆ i64       ┆ i64       ┆ str               ┆ str                │
╞════════════╪════════════════════╪═══════════╪═══════════╪═══════════════════╪════════════════════╡
│ nm9570225  ┆ Senarath           ┆ null      ┆ null      ┆ actor             ┆ tt7925724,tt790298 │
│            ┆ Mapitigama         ┆           ┆           ┆                   ┆ 2                  │
│ nm14116768 ┆ Danny Swart        ┆ null      ┆ null      ┆ null              ┆ tt0412116          │
│ nm9545314  ┆ Rachid Ouchen      ┆ null      ┆ null      ┆ composer          ┆ tt7855880          │
│ nm15880378 ┆ John Steward       ┆ null


Table "title.akas.tsv.gz" has Rows_MLN = 50.5 and columns = 8 :

Column "titleId" Type = "String" # Unique_MLN = 11.2 # Unique_pcts = 22.0% # Empty_pcts = 0.0% #  Prefix detected = tt # 
Column "ordering" Type = "Int64" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 0.0% # 
Column "title" Type = "String" # Unique_MLN = 7.2 # Unique_pcts = 14.0% # Empty_pcts = 0.0% # 
Column "region" Type = "String" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 22.0% # 
Column "language" Type = "String" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 33.0% # 
Column "types" Type = "String" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 69.0% # 
Column "attributes" Type = "String" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 99.0% # 
Column "isOriginalTitle" Type = "Int64" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 0.0% # 
Prefix detected : ['tt']
Primary Key is [] has size_MLN []
Foreign Key could be ['titleId'] has size_MLN [11.2]


<bound method DataFrame.transpose of shape: (6, 8)
┌────────────┬──────────┬───────────┬────────┬──────────┬──────────┬────────────┬──────────────────┐
│ titleId    ┆ ordering ┆ title     ┆ region ┆ language ┆ types    ┆ attributes ┆ isOriginalTitle  │
│ ---        ┆ ---      ┆ ---       ┆ ---    ┆ ---      ┆ ---      ┆ ---        ┆ ---              │
│ str        ┆ i64      ┆ str       ┆ str    ┆ str      ┆ str      ┆ str        ┆ i64              │
╞════════════╪══════════╪═══════════╪════════╪══════════╪══════════╪════════════╪══════════════════╡
│ tt11894814 ┆ 2        ┆ Episódio  ┆ PT     ┆ pt       ┆ null     ┆ null       ┆ 0                │
│            ┆          ┆ #1.167    ┆        ┆          ┆          ┆            ┆                  │
│ tt9763110  ┆ 4        ┆ Épisode   ┆ FR     ┆ fr       ┆ null     ┆ null       ┆ 0                │
│            ┆          ┆ #1.66     ┆        ┆          ┆          ┆            ┆                  │
│ tt21431636 ┆ 4        ┆ Épisode   ┆ FR


Table "title.basics.tsv.gz" has Rows_MLN = 11.2 and columns = 9 :

Column "tconst" Type = "String" # Unique_MLN = 11.2 # Unique_pcts = 100.0% # Empty_pcts = 0.0% #  Prefix detected = tt # 
Column "titleType" Type = "String" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 0.0% # 
Column "primaryTitle" Type = "String" # Unique_MLN = 5.0 # Unique_pcts = 45.0% # Empty_pcts = 0.0% # 
Column "originalTitle" Type = "String" # Unique_MLN = 5.1 # Unique_pcts = 45.0% # Empty_pcts = 0.0% # 
Column "isAdult" Type = "Int64" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 0.0% # 
Column "startYear" Type = "Int64" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 13.0% # 
Column "endYear" Type = "String" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 99.0% # 
Column "runtimeMinutes" Type = "Int64" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 68.0% # 
Column "genres" Type = "String" # /!\ List detected with ratio 162.0% /!\ Unique_MLN = 0.0 # Unique_pcts = 0.0% # 

...

<bound method DataFrame.transpose of shape: (6, 6)
┌────────────┬──────────┬────────────┬──────────┬──────────┬────────────────┐
│ tconst     ┆ ordering ┆ nconst     ┆ category ┆ job      ┆ characters     │
│ ---        ┆ ---      ┆ ---        ┆ ---      ┆ ---      ┆ ---            │
│ str        ┆ i64      ┆ str        ┆ str      ┆ str      ┆ str            │
╞════════════╪══════════╪════════════╪══════════╪══════════╪════════════════╡
│ tt1387328  ┆ 6        ┆ nm3351177  ┆ actor    ┆ null     ┆ ["Bubba"]      │
│ tt1379127  ┆ 9        ┆ nm0587583  ┆ director ┆ null     ┆ null           │
│ tt3497126  ┆ 3        ┆ nm6248991  ┆ producer ┆ producer ┆ null           │
│ tt15286038 ┆ 4        ┆ nm12871918 ┆ self     ┆ null     ┆ ["Themselves"] │
│ tt3822230  ┆ 6        ┆ nm5023462  ┆ self     ┆ null     ┆ ["Self"]       │
│ tt12885744 ┆ 3        ┆ nm3046606  ┆ actress  ┆ null     ┆ null           │
└────────────┴──────────┴────────────┴──────────┴──────────┴────────────────┘>


Table "title.ratings.tsv.gz" has Rows_MLN = 1.5 and columns = 3 :

Column "tconst" Type = "String" # Unique_MLN = 1.5 # Unique_pcts = 100.0% # Empty_pcts = 0.0% #  Prefix detected = tt # 
Column "averageRating" Type = "Float64" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 0.0% # 
Column "numVotes" Type = "Int64" # Unique_MLN = 0.0 # Unique_pcts = 2.0% # Empty_pcts = 0.0% # 
Prefix detected : ['tt']
Primary Key is ['tconst'] has size_MLN [1.5]
Foreign Key could be [] has size_MLN []


<bound method DataFrame.transpose of shape: (6, 3)
┌────────────┬───────────────┬──────────┐
│ tconst     ┆ averageRating ┆ numVotes │
│ ---        ┆ ---           ┆ ---      │
│ str        ┆ f64           ┆ i64      │
╞════════════╪═══════════════╪══════════╡
│ tt19712048 ┆ 8.4           ┆ 26       │
│ tt3253760  ┆ 8.0           ┆ 57       │
│ tt15147826 ┆ 8.9           ┆ 19       │
│ tt3982726  ┆ 7.9           ┆ 7        │
│ tt13852664 ┆ 6.5           ┆ 64       │
│ tt0333410  ┆ 7.1           ┆ 9        │
└────────────┴───────────────┴──────────┘>


Table "TMDB.csv" has Rows_MLN = 0.3 and columns = 25 :

Column "adult" Type = "Boolean" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 0.0% # 
Column "backdrop_path" Type = "String" # Unique_MLN = 0.2 # Unique_pcts = 49.0% # Empty_pcts = 51.0% # 
Column "budget" Type = "Int64" # Unique_MLN = 0.0 # Unique_pcts = 1.0% # Empty_pcts = 0.0% # 
Column "genres" Type = "String" # Unique_MLN = 0.0 # Unique_pcts = 3.0% # Empty_pcts = 0.0% # 
Column "homepage" Type = "String" # Unique_MLN = 0.0 # Unique_pcts = 14.0% # Empty_pcts = 86.0% #  Prefix detected = ht # 
Column "id" Type = "Int64" # Unique_MLN = 0.3 # Unique_pcts = 100.0% # Empty_pcts = 0.0% # 
Column "imdb_id" Type = "String" # Unique_MLN = 0.3 # Unique_pcts = 100.0% # Empty_pcts = 0.0% #  Prefix detected = tt # 
Column "original_language" Type = "String" # Unique_MLN = 0.0 # Unique_pcts = 0.0% # Empty_pcts = 0.0% # 
Column "original_title" Type = "String" # Unique_MLN = 0.3 # Unique_pcts = 92.0% # Empty_pcts = 0.0% # 
Column "

<bound method DataFrame.transpose of shape: (6, 25)
┌───────┬────────────┬────────┬────────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ adult ┆ backdrop_p ┆ budget ┆ genres     ┆ … ┆ vote_avera ┆ vote_count ┆ production ┆ production │
│ ---   ┆ ath        ┆ ---    ┆ ---        ┆   ┆ ge         ┆ ---        ┆ _companies ┆ _companies │
│ bool  ┆ ---        ┆ i64    ┆ str        ┆   ┆ ---        ┆ i64        ┆ _name      ┆ _country   │
│       ┆ str        ┆        ┆            ┆   ┆ f64        ┆            ┆ ---        ┆ ---        │
│       ┆            ┆        ┆            ┆   ┆            ┆            ┆ str        ┆ str        │
╞═══════╪════════════╪════════╪════════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ false ┆ null       ┆ 0      ┆ ['Drama']  ┆ … ┆ 0.0        ┆ 0          ┆ []         ┆ []         │
│ false ┆ /72q31NaxV ┆ 0      ┆ ['Family', ┆ … ┆ 5.475      ┆ 20         ┆ ['Walt     ┆ null       │
│       ┆ 8x6mPbiYkC ┆        ┆ 'Animat

In [4]:
infodict

{'name.basics.tsv.gz': {'name': 'name.basics.tsv.gz',
  'lines': 13,
  'cols': 6,
  'PKey': {'id': ['nconst'], 'Size': [13.9]},
  'FKey': {'id': ['knownForTitles_exploded'], 'Size': [5.8]},
  'PKD': {'id': ['name.basics.tsv.gz_nconst_nm_13.9']},
  'FKD': {'id': ['name.basics.tsv.gz_knownForTitles-exploded_tt_5.8']}},
 'title.akas.tsv.gz': {'name': 'title.akas.tsv.gz',
  'lines': 50,
  'cols': 8,
  'PKey': {'id': [], 'Size': []},
  'FKey': {'id': ['titleId'], 'Size': [11.2]},
  'PKD': {'id': []},
  'FKD': {'id': ['title.akas.tsv.gz_titleId_tt_11.2']}},
 'title.basics.tsv.gz': {'name': 'title.basics.tsv.gz',
  'lines': 11,
  'cols': 9,
  'PKey': {'id': ['tconst'], 'Size': [11.2]},
  'FKey': {'id': [], 'Size': []},
  'PKD': {'id': ['title.basics.tsv.gz_tconst_tt_11.2']},
  'FKD': {'id': []}},
 'title.crew.tsv.gz': {'name': 'title.crew.tsv.gz',
  'lines': 10,
  'cols': 3,
  'PKey': {'id': ['tconst'], 'Size': [10.6]},
  'FKey': {'id': ['directors_exploded', 'writers_exploded'],
   'Size': [

In [5]:
for ky in infodict.keys():
    print(ky)
    for kyy in infodict[ky]:
        print(kyy)

name.basics.tsv.gz
name
lines
cols
PKey
FKey
PKD
FKD
title.akas.tsv.gz
name
lines
cols
PKey
FKey
PKD
FKD
title.basics.tsv.gz
name
lines
cols
PKey
FKey
PKD
FKD
title.crew.tsv.gz
name
lines
cols
PKey
FKey
PKD
FKD
title.episode.tsv.gz
name
lines
cols
PKey
FKey
PKD
FKD
title.principals.tsv.gz
name
lines
cols
PKey
FKey
PKD
FKD
title.ratings.tsv.gz
name
lines
cols
PKey
FKey
PKD
FKD
TMDB.csv
name
lines
cols
PKey
FKey
PKD
FKD


In [6]:
pkdlook=[]
fkdlook=[]
for ky in infodict.keys():
    print(infodict[ky]['PKD']['id'])
    print(infodict[ky]['FKD']['id'])
    for kky in infodict[ky]['PKD']['id']:
        pkdlook.append(infodict[ky]['PKD']['id'])
    for kky in infodict[ky]['FKD']['id']:
        fkdlook.append(infodict[ky]['FKD']['id'])


['name.basics.tsv.gz_nconst_nm_13.9']
['name.basics.tsv.gz_knownForTitles-exploded_tt_5.8']
[]
['title.akas.tsv.gz_titleId_tt_11.2']
['title.basics.tsv.gz_tconst_tt_11.2']
[]
['title.crew.tsv.gz_tconst_tt_10.6']
['title.crew.tsv.gz_directors-exploded_nm_0.9', 'title.crew.tsv.gz_writers-exploded_nm_1.4']
['title.episode.tsv.gz_tconst_tt_8.6']
['title.episode.tsv.gz_parentTconst_tt_0.2']
[]
['title.principals.tsv.gz_tconst_tt_10.2', 'title.principals.tsv.gz_nconst_nm_6.5']
['title.ratings.tsv.gz_tconst_tt_1.5']
[]
['TMDB.csv_imdb_id_tt_0.3']
['TMDB.csv_backdrop_path_0.2', 'TMDB.csv_poster_path_0.3']


In [7]:
pkdlook

[['name.basics.tsv.gz_nconst_nm_13.9'],
 ['title.basics.tsv.gz_tconst_tt_11.2'],
 ['title.crew.tsv.gz_tconst_tt_10.6'],
 ['title.episode.tsv.gz_tconst_tt_8.6'],
 ['title.ratings.tsv.gz_tconst_tt_1.5'],
 ['TMDB.csv_imdb_id_tt_0.3']]

In [8]:
len(pkdlook)

6

In [9]:
pkdlook[1][0].split("_")

['title.basics.tsv.gz', 'tconst', 'tt', '11.2']

In [10]:
pkdlook[0][0].split("_")[2]

'nm'

In [11]:
pkdlook

[['name.basics.tsv.gz_nconst_nm_13.9'],
 ['title.basics.tsv.gz_tconst_tt_11.2'],
 ['title.crew.tsv.gz_tconst_tt_10.6'],
 ['title.episode.tsv.gz_tconst_tt_8.6'],
 ['title.ratings.tsv.gz_tconst_tt_1.5'],
 ['TMDB.csv_imdb_id_tt_0.3']]

In [12]:
pkfixlist=[]
for loop in range(len(pkdlook)):
    if pkdlook[loop][0].split("_")[-2] in pkfixlist:
        True
    else:
        pkfixlist.append(pkdlook[loop][0].split("_")[-2])

In [13]:
pkfixlist

['nm', 'tt']

In [16]:
pkdlook[1][0]

'title.basics.tsv.gz_tconst_tt_11.2'

In [17]:
ppfixfinal=[]
for ppfix in pkfixlist:
    countmax=-1.0
    indexmax=-1.0
    for loop in range(len(pkdlook)):
        if pkdlook[loop][0].split("_")[-2]==ppfix:
            if (float(pkdlook[loop][0].split("_")[-1])>float(countmax)):
                countmax=float(pkdlook[loop][0].split("_")[-1])
                pfixmax=loop
    ppfixfinal.append(pkdlook[pfixmax][0])

In [18]:
ppfixfinal

['name.basics.tsv.gz_nconst_nm_13.9', 'title.basics.tsv.gz_tconst_tt_11.2']

In [19]:
len(pkdlook)

6

In [20]:
from itertools import chain
fffixfinal=list(chain.from_iterable(fkdlook))

In [21]:
for pfix in list(chain.from_iterable(pkdlook)):
    if pfix in ppfixfinal:
        True
    else:
        fffixfinal.append(pfix)

In [22]:
ppfixfinal

['name.basics.tsv.gz_nconst_nm_13.9', 'title.basics.tsv.gz_tconst_tt_11.2']

In [23]:
fffixfinal

['name.basics.tsv.gz_knownForTitles-exploded_tt_5.8',
 'title.akas.tsv.gz_titleId_tt_11.2',
 'title.crew.tsv.gz_directors-exploded_nm_0.9',
 'title.crew.tsv.gz_writers-exploded_nm_1.4',
 'title.crew.tsv.gz_directors-exploded_nm_0.9',
 'title.crew.tsv.gz_writers-exploded_nm_1.4',
 'title.episode.tsv.gz_parentTconst_tt_0.2',
 'title.principals.tsv.gz_tconst_tt_10.2',
 'title.principals.tsv.gz_nconst_nm_6.5',
 'title.principals.tsv.gz_tconst_tt_10.2',
 'title.principals.tsv.gz_nconst_nm_6.5',
 'TMDB.csv_backdrop_path_0.2',
 'TMDB.csv_poster_path_0.3',
 'TMDB.csv_backdrop_path_0.2',
 'TMDB.csv_poster_path_0.3',
 'title.crew.tsv.gz_tconst_tt_10.6',
 'title.episode.tsv.gz_tconst_tt_8.6',
 'title.ratings.tsv.gz_tconst_tt_1.5',
 'TMDB.csv_imdb_id_tt_0.3']

In [24]:
fffixfinal=list(dict.fromkeys(fffixfinal))

In [25]:
ffindextable=[]
ffindexcol=[]
ffffixfinal=[]
for fffix in fffixfinal:
    if fffix.split("_")[-2] in pkfixlist:
        ffindextable.append(fffix.split("_")[0])
        ffindexcol.append(fffix.split("_")[1])
        ffffixfinal.append(fffix)




In [26]:
for ppfix in ppfixfinal[0:1]:
    nmtable=[]
    for fffix in ffffixfinal:
        if (ppfix.split("_")[-2]==fffix.split("_")[-2]):
            nmtable.append(str(round(100*float(fffix.split("_")[-1])/float(ppfix.split("_")[-1])))+'%')
        else:
            nmtable.append(np.nan)

In [27]:
for ppfix in ppfixfinal[1:2]:
    tttable=[]
    for fffix in ffffixfinal:
        if (ppfix.split("_")[-2]==fffix.split("_")[-2]):
            tttable.append(str(round(100*float(fffix.split("_")[-1])/float(ppfix.split("_")[-1])))+'%')
        else:
            tttable.append(np.nan)

In [30]:
import pandas as pd

In [31]:
dfnm=pd.DataFrame(data=nmtable, index=pd.MultiIndex.from_arrays([ffindextable, ffindexcol], names=('table', 'column')), columns=['Name'])
dftt=pd.DataFrame(data=tttable, index=pd.MultiIndex.from_arrays([ffindextable, ffindexcol], names=('table', 'column')), columns=['Movie'])

In [32]:
df_index=pd.concat([dftt, dfnm], axis=1)

# Structure finale

In [33]:
display(df_index.sort_index())

Movie Name
table                   column                            
TMDB.csv                imdb                       3%  NaN
name.basics.tsv.gz      knownForTitles-exploded   52%  NaN
title.akas.tsv.gz       titleId                  100%  NaN
title.crew.tsv.gz       directors-exploded        NaN   6%
                        tconst                    95%  NaN
                        writers-exploded          NaN  10%
title.episode.tsv.gz    parentTconst               2%  NaN
                        tconst                    77%  NaN
title.principals.tsv.gz nconst                    NaN  47%
                        tconst                    91%  NaN
title.ratings.tsv.gz    tconst                    13%  NaN

In [34]:
df_index.sort_index().transpose()

table  TMDB.csv      name.basics.tsv.gz title.akas.tsv.gz  title.crew.tsv.gz  \
column     imdb knownForTitles-exploded           titleId directors-exploded   
Movie        3%                     52%              100%                NaN   
Name        NaN                     NaN               NaN                 6%   

table                          title.episode.tsv.gz         \
column tconst writers-exploded         parentTconst tconst   
Movie     95%              NaN                   2%    77%   
Name      NaN              10%                  NaN    NaN   

table  title.principals.tsv.gz        title.ratings.tsv.gz  
column                  nconst tconst               tconst  
Movie                      NaN    91%                  13%  
Name                       47%    NaN                  NaN

In [36]:
fkdlook

[['name.basics.tsv.gz_knownForTitles-exploded_tt_5.8'],
 ['title.akas.tsv.gz_titleId_tt_11.2'],
 ['title.crew.tsv.gz_directors-exploded_nm_0.9',
  'title.crew.tsv.gz_writers-exploded_nm_1.4'],
 ['title.crew.tsv.gz_directors-exploded_nm_0.9',
  'title.crew.tsv.gz_writers-exploded_nm_1.4'],
 ['title.episode.tsv.gz_parentTconst_tt_0.2'],
 ['title.principals.tsv.gz_tconst_tt_10.2',
  'title.principals.tsv.gz_nconst_nm_6.5'],
 ['title.principals.tsv.gz_tconst_tt_10.2',
  'title.principals.tsv.gz_nconst_nm_6.5'],
 ['TMDB.csv_backdrop_path_0.2', 'TMDB.csv_poster_path_0.3'],
 ['TMDB.csv_backdrop_path_0.2', 'TMDB.csv_poster_path_0.3']]

In [37]:
len

<function len(obj, /)>

In [38]:
pkdlook[0]

['name.basics.tsv.gz_nconst_nm_13.9']

In [39]:
fkdlook

[['name.basics.tsv.gz_knownForTitles-exploded_tt_5.8'],
 ['title.akas.tsv.gz_titleId_tt_11.2'],
 ['title.crew.tsv.gz_directors-exploded_nm_0.9',
  'title.crew.tsv.gz_writers-exploded_nm_1.4'],
 ['title.crew.tsv.gz_directors-exploded_nm_0.9',
  'title.crew.tsv.gz_writers-exploded_nm_1.4'],
 ['title.episode.tsv.gz_parentTconst_tt_0.2'],
 ['title.principals.tsv.gz_tconst_tt_10.2',
  'title.principals.tsv.gz_nconst_nm_6.5'],
 ['title.principals.tsv.gz_tconst_tt_10.2',
  'title.principals.tsv.gz_nconst_nm_6.5'],
 ['TMDB.csv_backdrop_path_0.2', 'TMDB.csv_poster_path_0.3'],
 ['TMDB.csv_backdrop_path_0.2', 'TMDB.csv_poster_path_0.3']]

In [40]:
fkdlook[1][0]

'title.akas.tsv.gz_titleId_tt_11.2'

In [41]:
for loop in range(len(pkdlook)):
    print(f'Matching of Primary Key {pkdlook[loop][0]}')
    for loop2 in range (len(fkdlook)):
        if pkdlook[loop][0].split("_")[-2]==fkdlook[loop2][0].split("_")[-2]:
            print(f' PKey {pkdlook[loop][0].split("_")[1]} from {pkdlook[loop][0].split("_")[0]} has same prefix "{pkdlook[loop][0].split("_")[-2]}" than FKey {fkdlook[loop2][0].split("_")[1]} from {fkdlook[loop2][0].split("_")[0]}')
        if ((float(pkdlook[loop][0].split("_")[-1])/float(fkdlook[loop2][0].split("_")[-1])>0.99) and (float(pkdlook[loop][0].split("_")[-1])/float(fkdlook[loop2][0].split("_")[-1])<1.01)):
            print(f' PKey {pkdlook[loop][0].split("_")[1]} from {pkdlook[loop][0].split("_")[0]} has same size than FKey {fkdlook[loop2][0].split("_")[1]} from {fkdlook[loop2][0].split("_")[0]}')
        elif ((float(pkdlook[loop][0].split("_")[-1])/float(fkdlook[loop2][0].split("_")[-1])>0.9) and (float(pkdlook[loop][0].split("_")[-1])/float(fkdlook[loop2][0].split("_")[-1])<1.1)):
            print(f' PKey {pkdlook[loop][0].split("_")[1]} from {pkdlook[loop][0].split("_")[0]} has close size with FKey {fkdlook[loop2][0].split("_")[1]} from {fkdlook[loop2][0].split("_")[0]} Deltasize_MLN ={round(abs((float(pkdlook[loop][0].split("_")[-1])-float(fkdlook[loop2][0].split("_")[-1]))),2)}')
        
#        print(pkdlook[loop][0].split("_")[-1])
#        print(fkdlook[loop2][0].split("_")[-1])
#        print(pkdlook[loop][0].split("_")[-2])
#        print(fkdlook[loop2][0].split("_")[-2])
#        print(pkdlook[loop][0].split("_")[1])
#        print(fkdlook[loop2][0].split("_")[1])
#        print(pkdlook[loop][0].split("_")[0])
#        print(fkdlook[loop2][0].split("_")[0])

Matching of Primary Key name.basics.tsv.gz_nconst_nm_13.9
 PKey nconst from name.basics.tsv.gz has same prefix "nm" than FKey directors-exploded from title.crew.tsv.gz
 PKey nconst from name.basics.tsv.gz has same prefix "nm" than FKey directors-exploded from title.crew.tsv.gz
Matching of Primary Key title.basics.tsv.gz_tconst_tt_11.2
 PKey tconst from title.basics.tsv.gz has same prefix "tt" than FKey knownForTitles-exploded from name.basics.tsv.gz
 PKey tconst from title.basics.tsv.gz has same prefix "tt" than FKey titleId from title.akas.tsv.gz
 PKey tconst from title.basics.tsv.gz has same size than FKey titleId from title.akas.tsv.gz
 PKey tconst from title.basics.tsv.gz has same prefix "tt" than FKey parentTconst from title.episode.tsv.gz
 PKey tconst from title.basics.tsv.gz has same prefix "tt" than FKey tconst from title.principals.tsv.gz
 PKey tconst from title.basics.tsv.gz has close size with FKey tconst from title.principals.tsv.gz Deltasize_MLN =1.0
 PKey tconst from titl